# Avito Demand Prediction Challenge

## Equipe:

* Catherine dos Santos Tostes - Aluna de Mestrado PPGI
* Jean
* Felipe

Este relatório tem como objetivo descrever as feautures do arquivo processado, descrever as estratégias no pré-processamento e na classificação dos dados, mostrando as diferenças entre os classificadores utilizados. Este trabalho foi realizado através da participação em uma competição do Kaggle da Avito Demand Prediction Challenge.

URL da Competição: https://www.kaggle.com/c/avito-demand-prediction

A Avito é um dos maiores sites de anúncios de produtos da Rússia com seções de categorias de diversos segmentos. O desafio da Avito trata de tentar prever se um produto irá ter uma alta demanda ou uma baixa demanda para que os vendedores saibam como otimizar as vendas e indicar aos vendedores o quanto de interesse o produto possa ter, para isso será feito uma análise do conteúdo da descrição dos produtos, aplicação de estratégias e classificadores para mostrar a diferença entre eles.

A classificação é uma forma de aplicar a aprendizagem supervisionada (Supervised Learning), ou seja, o algoritmo será treinado através de um conjunto de dados de entrada e irá classificar os dados, através do treinamento o algoritmo irá tentar prever os resultados.

## Importando módulos e Bibliotecas

Será importado os módulos e bibliotecas que serão necessários para a leitura do arquivo, para o processamento dos dados, a aplicação de métodos e classificadores, cálculos necessários para avaliação e criação de gráficos representativos do problema.

In [25]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

Iremos carregar e ler o arquivo de treinamento que está disponível com o nome de train.csv.zip, será utilizado o DataFrame do Pandas, que é uma biblioteca para auxiliar na análise de dados tabulares bidimensionais.

In [3]:
dataset_train = pd.read_csv('train.csv')

Com o arquivo de treinamento carregado, para se entender os dados será utilizado uma função head() do Pandas para exibir as 5 primeiras linhas do arquivo.

In [4]:
dataset_train.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264,0.20797


Com a leitura das 5 linhas do arquivo, pode-se notar que ele está dividido em 18 colunas:

* item_id: Contém a String de código de cada item à venda.
* user_id: Contém a String de código de cada usuário.
* region: De qual região da Rússia o usuário faz parte.
* city: De qual cidade da Rússia o usuário faz parte.
* parent_category_name: Nome da categoria que o produto faz parte.
* category_name: Nome da sub-categoria que o produto faz parte.
* param_1: Tipo do produto.
* param_2: Tipo do produto.
* param_3: Tipo do produto.
* title: Título do produto à venda.
* description: Descrição do produto à venda.
* price: Preço do produto.
* item_seq_number: Número de sequência do produto.
* activation_date: Data de divulgação do produto.
* user_type: Tipo de usuário (se é pessoal ou empresarial).
* image: Imagem.
* image_top_1: Tamanho da Imagem.
* deal_probability: Probabilidade de venda do produto.

Para entender mais sobre os dados será realizado a contagem das linhas e colunas respectivamente na base de dados.

In [5]:
dataset_train.shape

(1503424, 18)

Contagem dos dados não-nulos de cada categoria:

In [6]:
dataset_train.count()

item_id                 1503424
user_id                 1503424
region                  1503424
city                    1503424
parent_category_name    1503424
category_name           1503424
param_1                 1441848
param_2                  848882
param_3                  640859
title                   1503424
description             1387148
price                   1418062
item_seq_number         1503424
activation_date         1503424
user_type               1503424
image                   1390836
image_top_1             1390836
deal_probability        1503424
dtype: int64

## Pré-Processamento

Alguns modelos de treinamento precisam ser pré-processados para serem normalizados antes de se realizar algum tipo de análise, para que a aplicação dos métodos de classificação não sejam feitos de forma ineficiente, gerando resultados não satisfatórios.

A padronização será realizada por partes.

Primeiramente iremos colocar todas as palavras presentes no título como minúsculas, para que não haja diferenças das mesmas palavras.

In [7]:
words_title = []

for word in dataset_train['title']:
    words_title.append(word.lower())

words_title

Com todas as palavras em minúsculo dentro de uma lista, será retirado do texto todos os stopwords (palavras presentes que não tem relevância para o texto ao qual está inserido) através do módulo nltk será importado para uso uma biblioteca em russo com as palavras já armazenadas.

In [21]:
filtered_words = [word for word in words_title if word not in stopwords.words('russian')]

Agora os dados do título do produto serão tratados através do módulo feature_extraction.text do Scikit-Learn, importando o TfidfVectorizer.

O TfidfVectorizer irá transformar as palavras do título para uma forma que os algoritmos de aprendizado de máquina reconhecem, pois eles esperam vetores numéricos, o texto será transformado em uma matriz, e será feito a técnica de bag_of_words que receberá a transformação dos dados de texto para matriz.

Este módulo funciona da seguinte maneira:
Com a frequência de cada String individual é tratado e é criado um vetor de frequências para um documento, criando uma amostra multivariada. Transformar uma coleção de documentos em vetores numéricos é chamado de vetorização. Essa estratégia é chamada de Bag of Words ou Bag of n-grams.

Cada frequência de ocorrência de token individual (normalizada ou não) é tratada como um recurso.

In [23]:
vector = TfidfVectorizer()
bag_of_words = vector.fit_transform(filtered_words)

Após essa transformação será gerada uma matriz com as feautures do TfidfVectorizer para cada documento, no caso do nosso problema é o título. Esta matriz é representada pelo número de títulos de produtos presentes na base de dados e a quantidade de títulos únicos presentes no vocabulário.

In [24]:
bag_of_words.shape

(1503413, 206935)

## Aplicação dos Classificadores

Com o pré-processamento dos dados a serem utilizados, a representação dos documentos agora como uma matriz de frequência de ocorrências, o algoritmo de aprendizado de máquina poderá aplicar classificadores para o treinamento do modelo a ser utilizado.

Foram escolhidos para serem utilizados os seguintes classificadores:
* Rede Neural
* Naive Bayes
* Support Vector Machine (SVM)

### Rede Neural